# Amazon SageMaker Model Governance - unified model card and model registry

This notebook walks you through the features of Amazon SageMaker Model Registry. For more information, see [Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-models.html) in the _Amazon SageMaker Developer Guide_.

Amazon SageMaker integrates Model Cards into Model Registry, making it easier for customers to manage governance information for specific model versions directly in Model Registry in just a few clicks. Customers register ML models in Model Registry to manage their models. Additionally, they can register ML model versions early in the development lifecycle, including essential business details and technical metadata. This integration allows customers to seamlessly review and govern models across their lifecycle from a single place. Customers have greater visibility into the model lifecycle from experimentation and training to evaluation and deployment. This streamlined experience ensures that model governance is consistent and easily accessible throughout the development.

In this example, you create a binary classification model along with persisting model card information into registry to manage unified governance information and view

## Notebook Overview

This notebook shows how to:

* Define a set of Pipeline parameters that can be used to parametrize a SageMaker Pipeline.

* Define a Processing step that performs cleaning, feature engineering, and splitting the input data into train and test data sets.

* Define a Training step that trains a model on the preprocessed train data set.

* Define a Processing step that evaluates the trained model's performance on the test dataset.

* Define a Register Model step that creates a model package from the estimator and model 
artifacts used to train the model.

* Define a Conditional step that measures a condition based on output from prior steps and conditionally executes other steps.

# Update the Sagemaker Python SDK to latest

In [1]:
!pip install --upgrade "sagemaker>=2"

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/conda/lib/python3.10/site-packages/pytz-2023.3.dist-info/METADATA'



In [2]:
!pip install --upgrade sagemaker-core

# Create Datazone project
This is optional step, however encouraged to help with governance. Amazon DataZone is a data management service that makes it faster and easier for you to catalog, discover, share, and govern data stored across AWS, on-premises, and third-party sources. With Amazon DataZone, administrators who oversee organization’s data assets can manage and govern access to data using fine-grained controls. These controls help ensure access with the right level of privileges and context. Amazon DataZone makes it easy for engineers, data scientists, product managers, analysts, and business users to share and access data throughout an organization so they can discover, use, and collaborate to derive data-driven insights.

First, [create a new datazone domain](https://us-east-1.console.aws.amazon.com/datazone/home?region=us-east-1#/) if it doesnt already exists.Then, [create a new DataZone Project](https://docs.aws.amazon.com/datazone/latest/userguide/create-new-project.html). Use the project id in the below check and run 

In [3]:
# Capture project id from DataZone project
project_id = "5rn1teh0tv85rb"

## Usecase and Dataset

In this tutorial, you will assume the role of a machine learning developer working at a bank. You have been asked to develop a machine learning model to predict whether a customer will enroll for a certificate of deposit (CD). The model will be trained on the marketing dataset that contains information on customer demographics, responses to marketing events, and external factors.

The data has been labeled for your convenience and a column in the dataset identifies whether the customer is enrolled for a product offered by the bank. A version of this dataset is publicly available from the ML repository curated by the University of California, Irvine. This tutorial implements a supervised machine learning model, since the data is labeled. (Unsupervised learning occurs when the datasets are not labeled.)


## Pipelines Instance
A SageMaker Pipeline instance is composed of three components:
    1. Name
    2. Parameters
    3. Steps

## Initial set up

In [4]:
# Setup
import boto3
import sagemaker

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
model_package_group_name = f"DirectMarketingExamplePackageGroupName"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


Upload the data into the default bucket. You can select your own data set for the `input_data_uri` as is appropriate.

In [5]:
local_path = "bank-additional-full.csv"

base_uri = f"s3://{default_bucket}/marketing"
input_data_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_path, 
    desired_s3_uri=base_uri,
)
print(input_data_uri)

s3://sagemaker-us-east-1-340280328827/marketing/bank-additional-full.csv


### Define Parameters to Parametrize Pipeline Execution

Define Pipeline parameters that you can use to parametrize the pipeline. Parameters enable custom pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - represents a `str` Python type
* `ParameterInteger` - represents an `int` Python type
* `ParameterFloat` - represents a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

The parameters defined in this workflow include:

* `processing_instance_type` - The `ml.*` instance type of the processing job.
* `processing_instance_count` - The instance count of the processing job.
* `training_instance_type` - The `ml.*` instance type of the training job.
* `model_approval_status` - What approval status to register the trained model with for CI/CD purposes ( "PendingManualApproval" is the default).
* `input_data` - The S3 bucket URI location of the input data

In [6]:
# Define pipeline parameters
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)
training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)
input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)

### Define a Processing Step for Feature Engineering

First, develop a preprocessing script that is specified in the Processing step.

This notebook cell writes a file `preprocessing.py`, which contains the preprocessing script. You can update the script, and rerun this cell to overwrite. The preprocessing script uses `scikit-learn` to do the following:

* It creates new variables: no_previous_contact, not_working
* It drops columns that are not necessary
* It one-hot encodes categorical variables

The Processing step executes the script on the input data. The Training step uses the preprocessed training features and labels to train a model. The Evaluation step uses the trained model and preprocessed test features and labels to evaluate the model.

In [7]:
!mkdir -p marketing

In [8]:
%%writefile marketing/preprocessing.py
import argparse
import os

import numpy as np
import pandas as pd

# Helper method
def process(df):
    # Add two new indicators
    df["no_previous_contact"] = (df["pdays"] == 999).astype(int)
    df["not_working"] = df["job"].isin(["student", "retired", "unemployed"]).astype(int)
    columns = list(df.columns)
    
    toremove = ["emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"]
    columns = [x for x in columns if x not in toremove]
    
    # Keeping only columns that we need
    df = df[columns]
    
    # One hot encode
    df=pd.get_dummies(df)
    df = pd.concat([df['y_yes'], df.drop(['y_no', 'y_yes'], axis=1)], axis=1)
    df = df.sample(frac=1).reset_index(drop=True)
    return df

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--input-path", type=str, default="/opt/ml/processing")
    args, _ = parser.parse_known_args()
    
    base_dir = args.input_path

    df = pd.read_csv(
        f"{base_dir}/input/bank-additional-full.csv",
        header=0
    )
    
    # Call the helper method
    df = process(df)
    
    train, validation, test = np.split(df, [int(.7*len(df)), int(.85*len(df))])

    train.to_csv(f"{base_dir}/train/train.csv", header=False, index=False)
    validation.to_csv(f"{base_dir}/validation/validation.csv", header=False, index=False)
    test.to_csv(f"{base_dir}/test/test.csv", header=False, index=False)

Overwriting marketing/preprocessing.py


Next, create an instance of an `SKLearnProcessor` processor and use that in our `ProcessingStep`.

You also specify the `framework_version` to use throughout this notebook.

Note the `processing_instance_type` and `processing_instance_count` parameters used by the processor instance.

In [9]:
from sagemaker.sklearn.processing import SKLearnProcessor


framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-marketing-process",
    role=role,
)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is interpreted in pipeline execution time only. As the function needs to evaluate the argument value in SDK compile time, the default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


Finally, use the processor instance to construct a `ProcessingStep`, along with the input and output channels, and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Note the `input_data` parameters passed into `ProcessingStep` is the input data used in the step. This input data is used by the processor instance when it is run.

Also, note the `"train_data"` and `"test_data"` named channels specified in the output configuration for the processing job. Step `Properties` can be used in subsequent steps and resolve to their runtime values at execution. Specifically, this usage is called out when you define the training step.

In [10]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    

step_process = ProcessingStep(
    name="MarketingProcess",
    processor=sklearn_processor,
    inputs=[
      ProcessingInput(source=input_data, destination="/opt/ml/processing/input"),  
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test")
    ],
    code="marketing/preprocessing.py",
)

### Define a Training Step to Train a Model

In this section, use Amazon SageMaker's [XGBoost Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) to train on this dataset. Configure an Estimator for the XGBoost algorithm and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later.

The model path where the models from training will be saved is also specified.

Note the `training_instance_type` parameter may be used in multiple places in the pipeline. In this case, the `training_instance_type` is passed into the estimator.

In [11]:
from sagemaker.estimator import Estimator

model_path = f"s3://{default_bucket}/MarketingTrain"
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=training_instance_type,
)
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path,
    role=role,
)
xgb_train.set_hyperparameters(
    objective="binary:logistic",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0
)

The input argument instance_type of function (sagemaker.image_uris.retrieve) is a pipeline variable (<class 'sagemaker.workflow.parameters.ParameterString'>), which is interpreted in pipeline execution time only. As the function needs to evaluate the argument value in SDK compile time, the default_value of this Parameter object will be used to override it. Please make sure the default_value is valid.


Finally, use the estimator instance to construct a `TrainingStep` as well as the `properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that's executed when the pipeline invokes the pipeline execution. This is similar to an estimator's `fit` method in the Python SDK.

Pass in the `S3Uri` of the `"train_data"` output channel to the `TrainingStep`. Also, use the other `"test_data"` output channel for model evaluation in the pipeline. The `properties` attribute of a Pipeline step matches the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [12]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="MarketingTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv"
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv"
        )
    },
)

### Define a Model Evaluation Step to Evaluate the Trained Model

First, develop an evaluation script that is specified in a Processing step that performs the model evaluation.

After pipeline execution, you can examine the resulting `evaluation.json` for analysis.

The evaluation script uses `xgboost` to do the following:

* Load the model.
* Read the test data.
* Issue predictions against the test data.
* Build a classification report, including accuracy and ROC curve.
* Save the evaluation report to the evaluation directory.

In [13]:
%%writefile marketing/evaluation.py
import os
import json
import pathlib
import pickle
import tarfile

import pandas as pd
import xgboost

import joblib
import numpy as np
import pandas as pd
import xgboost

from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

if __name__ == "__main__":
    model_path = f"/opt/ml/processing/model/model.tar.gz"
    with tarfile.open(model_path) as tar:
        tar.extractall(path=".")
    
    model = pickle.load(open("xgboost-model", "rb"))

    print("Loading test input data")
    test_path = "/opt/ml/processing/test/test.csv"
    df = pd.read_csv(test_path, header=None)
    
    y_test = df.iloc[:, 0].to_numpy()
    df.drop(df.columns[0], axis=1, inplace=True)
    
    X_test = xgboost.DMatrix(df.values)
    
    predictions = model.predict(X_test)

    print("Creating classification evaluation report")
    acc = accuracy_score(y_test, predictions.round())
    auc = roc_auc_score(y_test, predictions.round())
    
    # The metrics reported can change based on the model used, 
    # but it must be a specific name per 
    # (https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-model-quality-metrics.html)
    report_dict = {
        "binary_classification_metrics": {
            "accuracy": {
                "value": acc,
                "standard_deviation": "NaN",
            },
            "auc": {"value": auc, "standard_deviation": "NaN"},
        },
    }
  
    print("Classification report:\n{}".format(report_dict))

    evaluation_output_path = os.path.join("/opt/ml/processing/evaluation", "evaluation.json")
    print("Saving classification report to {}".format(evaluation_output_path))

    with open(evaluation_output_path, "w") as f:
        f.write(json.dumps(report_dict))

Overwriting marketing/evaluation.py


Next, create an instance of a `ScriptProcessor` processor and use it in the `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [14]:
from sagemaker.processing import ScriptProcessor


script_eval = ScriptProcessor(
    image_uri=image_uri,
    command=["python3"],
    instance_type=processing_instance_type,
    instance_count=1,
    base_job_name="script-marketing-eval",
    role=role,
)

Use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Specifically, the `S3ModelArtifacts` from the `step_train` `properties` and the `S3Uri` of the `"test_data"` output channel of the `step_process` `properties` are passed into the inputs. The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [15]:
from sagemaker.workflow.properties import PropertyFile


evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json"
)
step_eval = ProcessingStep(
    name="MarketingEval",
    processor=script_eval,
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test"
        )
    ],
    outputs=[
        ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation"),
    ],
    code="marketing/evaluation.py",
    property_files=[evaluation_report],
)

### Define a Register Model Step to Create a Model Package

Use the estimator instance specified in the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a model package. A model package is a reusable model artifacts abstraction that packages all ingredients required for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A model package group is a collection of model packages. A model package group can be created for a specific ML business problem, and new versions of the model packages can be added to it. Typically, customers are expected to create a ModelPackageGroup for a SageMaker pipeline so that model package versions can be added to the group for every SageMaker Pipeline run.

The construction of `RegisterModel` is similar to an estimator instance's `register` method in the Python SDK.

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Note that the specific model package group name provided in this notebook can be used in the model registry and CI/CD work with SageMaker Projects.

In [16]:
import io
import os
import numpy as np
from six.moves.urllib.parse import urlparse
from pprint import pprint
import boto3
import sagemaker
from sagemaker.image_uris import retrieve
import sagemaker.amazon.common as smac
from sagemaker.model_card import (
    ModelCard,
    ModelOverview,
    IntendedUses,
    BusinessDetails,
    AdditionalInformation,
    ModelCardStatusEnum,
    ObjectiveFunctionEnum,
    RiskRatingEnum,
    MetricTypeEnum,
    TrainingDetails
)
from sagemaker.model import ModelPackage as MP

In [17]:
model_overview = ModelOverview(
    #model_description="This is an example model used for a Python SDK demo of unified Amazon SageMaker Model Registry and Model Cards.",
    #problem_type="Binary Classification",
    #algorithm_type="Logistic Regression",
    model_creator="DEMO-Model-Registry-ModelCard-Unification",
    #model_owner="datascienceteam",
)

In [18]:
intended_uses = IntendedUses(
    purpose_of_model="Test model card.",
    intended_uses="Not used except this test.",
    factors_affecting_model_efficiency="No.",
    risk_rating=RiskRatingEnum.LOW,
    explanations_for_risk_rating="Just an example.",
)
business_details = BusinessDetails(
    business_problem="The business problem that your model is used to solve.",
    business_stakeholders="The stakeholders who have the interest in the business that your model is used for.",
    line_of_business="Services that the business is offering.",
)
additional_information = AdditionalInformation(
    ethical_considerations="Your model ethical consideration.",
    caveats_and_recommendations="Your model's caveats and recommendations.",
    custom_details={"Datazone projects": project_id},
)

In [19]:
my_card = ModelCard(
    name="MyModelCard",
    status=ModelCardStatusEnum.DRAFT,
    model_overview=model_overview,
    intended_uses=intended_uses,
    business_details=business_details,
    additional_information=additional_information,
    sagemaker_session=sagemaker_session,
)

In [20]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics 
from sagemaker.workflow.step_collections import RegisterModel


model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
        ),
        content_type="application/json"
    )
)
step_register = RegisterModel(
    name="MarketingRegisterModel",
    estimator=xgb_train,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
    model_card=my_card
)

Popping out 'ProcessingJobName' from the pipeline definition by default since it will be overridden at pipeline execution time. Please utilize the PipelineDefinitionConfig to persist this field in the pipeline definition if desired.


### Define a Condition Step to Check Accuracy and Conditionally Register a Model in the Model Registry

In this step, the model is registered only if the accuracy of the model, as determined by the evaluation step `step_eval`, exceeded a specified value. A `ConditionStep` enables pipelines to support conditional execution in the pipeline DAG based on the conditions of the step properties. 

In the following section, you:

* Define a `ConditionLessThanOrEqualTo` on the accuracy value found in the output of the evaluation step, `step_eval`.
* Use the condition in the list of conditions in a `ConditionStep`.
* Pass the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`, which are only executed, if the condition evaluates to `True`.

In [21]:
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)

cond_lte = ConditionGreaterThanOrEqualTo(  # You can change the condition here
        left=JsonGet(
            step=step_eval,
            property_file=evaluation_report,
            json_path="binary_classification_metrics.accuracy.value",  # This should follow the structure of your report_dict defined in the evaluate.py file.
        ),
        right=0.8,  # You can change the threshold here
)

step_cond = ConditionStep(
    name="MarketingAccuracyCond",
    conditions=[cond_lte],
    if_steps=[step_register],
    else_steps=[]
)

The class JsonGet has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


### Define a Pipeline of Parameters, Steps, and Conditions

In this section, combine the steps into a Pipeline so it can be executed.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair.

Note:

* All of the parameters used in the definitions must be present.
* Steps passed into the pipeline do not have to be listed in the order of execution. The SageMaker Pipeline service resolves the _data dependency_ DAG as steps for the execution to complete.
* Steps must be unique to across the pipeline step list and all condition step if/else lists.

In [22]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = f"ExamplePipeline"
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,
        model_approval_status,
        input_data,
    ],
    steps=[step_process, step_train, step_eval, step_cond],
)

#### (Optional) Examining the pipeline definition

The JSON of the pipeline definition can be examined to confirm the pipeline is well-defined and the parameters and step properties resolve correctly.

In [23]:
import json

definition = json.loads(pipeline.definition())
definition

{'Version': '2020-12-01',
 'Metadata': {},
 'Parameters': [{'Name': 'ProcessingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ProcessingInstanceCount', 'Type': 'Integer', 'DefaultValue': 1},
  {'Name': 'TrainingInstanceType',
   'Type': 'String',
   'DefaultValue': 'ml.m5.xlarge'},
  {'Name': 'ModelApprovalStatus',
   'Type': 'String',
   'DefaultValue': 'PendingManualApproval'},
  {'Name': 'InputData',
   'Type': 'String',
   'DefaultValue': 's3://sagemaker-us-east-1-340280328827/marketing/bank-additional-full.csv'}],
 'PipelineExperimentConfig': {'ExperimentName': {'Get': 'Execution.PipelineName'},
  'TrialName': {'Get': 'Execution.PipelineExecutionId'}},
 'Steps': [{'Name': 'MarketingProcess',
   'Type': 'Processing',
   'Arguments': {'ProcessingResources': {'ClusterConfig': {'InstanceType': {'Get': 'Parameters.ProcessingInstanceType'},
      'InstanceCount': {'Get': 'Parameters.ProcessingInstanceCount'},
      'VolumeSizeInGB': 30}},
    'AppSpe

### Submit the pipeline to SageMaker and start execution

Submit the pipeline definition to the Pipeline service. The role passed in will be used by the Pipeline service to create all the jobs defined in the steps.

In [24]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:340280328827:pipeline/ExamplePipeline',
 'ResponseMetadata': {'RequestId': '966bebd5-7bb6-4aac-9e58-7c97544700ae',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '966bebd5-7bb6-4aac-9e58-7c97544700ae',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '83',
   'date': 'Mon, 23 Sep 2024 17:31:05 GMT'},
  'RetryAttempts': 0}}

Start the pipeline and accept all of the default parameters.

In [25]:
execution = pipeline.start()

### Pipeline Operations: Examining and Waiting for Pipeline Execution

Describe the pipeline execution.

In [26]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:340280328827:pipeline/ExamplePipeline',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:340280328827:pipeline/ExamplePipeline/execution/qs4nfbqgsscf',
 'PipelineExecutionDisplayName': 'execution-1727112665835',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2024, 9, 23, 17, 31, 5, 733000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 9, 23, 17, 31, 5, 733000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:340280328827:user-profile/d-ykydvorozt8h/default-1710454128270',
  'UserProfileName': 'default-1710454128270',
  'DomainId': 'd-ykydvorozt8h',
  'IamIdentity': {'Arn': 'arn:aws:sts::340280328827:assumed-role/CFN-SM-IM-Lambda-Catalog-SageMakerExecutionRole-1TSLA2QB1DJKS/SageMaker',
   'PrincipalId': 'AROAU6OSJYJ5VWYPHAFI6:SageMaker'}},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:340280328827:user-profile/d-ykydvorozt8h/default-171045

Wait for the execution to complete.

In [27]:
execution.wait()

List the steps in the execution. These are the steps in the pipeline that have been resolved by the step executor service.

In [28]:
execution.list_steps()

[{'StepName': 'MarketingRegisterModel-RegisterModel',
  'StartTime': datetime.datetime(2024, 9, 23, 17, 39, 8, 150000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 9, 23, 17, 39, 10, 333000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:340280328827:model-package/DirectMarketingExamplePackageGroupName/1'}},
  'AttemptCount': 1},
 {'StepName': 'MarketingAccuracyCond',
  'StartTime': datetime.datetime(2024, 9, 23, 17, 39, 7, 310000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 9, 23, 17, 39, 7, 755000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Condition': {'Outcome': 'True'}},
  'AttemptCount': 1},
 {'StepName': 'MarketingEval',
  'StartTime': datetime.datetime(2024, 9, 23, 17, 36, 30, 513000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 9, 23, 17, 39, 6, 100000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagem

### Examining the Evalution

Examine the resulting model evaluation after the pipeline completes. Download the resulting `evaluation.json` file from S3 and print the report.

In [29]:
from pprint import pprint


evaluation_json = sagemaker.s3.S3Downloader.read_file("{}/evaluation.json".format(
    step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
))
pprint(json.loads(evaluation_json))

{'binary_classification_metrics': {'accuracy': {'standard_deviation': 'NaN',
                                                'value': 0.9101796407185628},
                                   'auc': {'standard_deviation': 'NaN',
                                           'value': 0.7215769660516066}}}


### Lineage

Review the lineage of the artifacts generated by the pipeline.

In [30]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

{'StepName': 'MarketingProcess', 'StartTime': datetime.datetime(2024, 9, 23, 17, 31, 7, 486000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 9, 23, 17, 34, 4, 807000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:340280328827:processing-job/pipelines-qs4nfbqgsscf-MarketingProcess-Ktj0pD3TmZ'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...8e8eb15a8a46/input/code/preprocessing.py,Input,DataSet,ContributedTo,artifact
1,s3://...28827/marketing/bank-additional-full.csv,Input,DataSet,ContributedTo,artifact
2,68331...om/sagemaker-scikit-learn:0.23-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...s4nfbqgsscf/MarketingProcess/output/test,Output,DataSet,Produced,artifact
4,s3://...gsscf/MarketingProcess/output/validation,Output,DataSet,Produced,artifact
5,s3://...4nfbqgsscf/MarketingProcess/output/train,Output,DataSet,Produced,artifact


{'StepName': 'MarketingTrain', 'StartTime': datetime.datetime(2024, 9, 23, 17, 34, 6, 272000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 9, 23, 17, 36, 29, 693000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'TrainingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:340280328827:training-job/pipelines-qs4nfbqgsscf-MarketingTrain-Y0wnnyFc4z'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...gsscf/MarketingProcess/output/validation,Input,DataSet,ContributedTo,artifact
1,s3://...4nfbqgsscf/MarketingProcess/output/train,Input,DataSet,ContributedTo,artifact
2,68331...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
3,s3://...tingTrain-Y0wnnyFc4z/output/model.tar.gz,Output,Model,Produced,artifact


{'StepName': 'MarketingEval', 'StartTime': datetime.datetime(2024, 9, 23, 17, 36, 30, 513000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 9, 23, 17, 39, 6, 100000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:us-east-1:340280328827:processing-job/pipelines-qs4nfbqgsscf-MarketingEval-oqyhKFL7Zk'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...c655e1c275ac100/input/code/evaluation.py,Input,DataSet,ContributedTo,artifact
1,s3://...s4nfbqgsscf/MarketingProcess/output/test,Input,DataSet,ContributedTo,artifact
2,s3://...tingTrain-Y0wnnyFc4z/output/model.tar.gz,Input,Model,ContributedTo,artifact
3,68331...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
4,s3://...62a4b00c655e1c275ac100/output/evaluation,Output,DataSet,Produced,artifact


{'StepName': 'MarketingAccuracyCond', 'StartTime': datetime.datetime(2024, 9, 23, 17, 39, 7, 310000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 9, 23, 17, 39, 7, 755000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'Condition': {'Outcome': 'True'}}, 'AttemptCount': 1}


None

{'StepName': 'MarketingRegisterModel-RegisterModel', 'StartTime': datetime.datetime(2024, 9, 23, 17, 39, 8, 150000, tzinfo=tzlocal()), 'EndTime': datetime.datetime(2024, 9, 23, 17, 39, 10, 333000, tzinfo=tzlocal()), 'StepStatus': 'Succeeded', 'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:340280328827:model-package/DirectMarketingExamplePackageGroupName/1'}}, 'AttemptCount': 1}


,Name/Source,Direction,Type,Association Type,Lineage Type
0,s3://...tingTrain-Y0wnnyFc4z/output/model.tar.gz,Input,Model,ContributedTo,artifact
1,68331...naws.com/sagemaker-xgboost:1.0-1-cpu-py3,Input,Image,ContributedTo,artifact
2,DirectMarketingExamplePackageGroupName-1-Pendi...,Input,Approval,ContributedTo,action
3,DirectMarketingExamplePackageGroupName-1727113...,Output,ModelGroup,AssociatedWith,context


### Parametrized Executions

You can run additional executions of the pipeline and specify different pipeline parameters. The parameters argument is a dictionary containing parameter names, and where the values are used to override the defaults values.

Based on the performance of the model, you might want to kick off another pipeline execution on a compute-optimized instance type and set the model approval status to "Approved" automatically. This means that the model package version generated by the `RegisterModel` step is automatically ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [31]:
execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [32]:
execution.wait()

In [33]:
execution.list_steps()

[{'StepName': 'MarketingRegisterModel-RegisterModel',
  'StartTime': datetime.datetime(2024, 9, 23, 17, 47, 32, 254000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 9, 23, 17, 47, 34, 441000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'RegisterModel': {'Arn': 'arn:aws:sagemaker:us-east-1:340280328827:model-package/DirectMarketingExamplePackageGroupName/2'}},
  'AttemptCount': 1},
 {'StepName': 'MarketingAccuracyCond',
  'StartTime': datetime.datetime(2024, 9, 23, 17, 47, 31, 146000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 9, 23, 17, 47, 31, 791000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Condition': {'Outcome': 'True'}},
  'AttemptCount': 1},
 {'StepName': 'MarketingEval',
  'StartTime': datetime.datetime(2024, 9, 23, 17, 44, 56, 945000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2024, 9, 23, 17, 47, 30, 570000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:s